<a href="https://colab.research.google.com/github/shirong52/LLM_study/blob/main/%E5%9B%BE%E5%83%8F%E7%AF%A1%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
from PIL.ExifTags import TAGS

def check_metadata_pure(image_path):
    img = Image.open(image_path)
    exif_data = img.getexif()
    if not exif_data:
        print("❌ 无 EXIF 信息，可能被清理")
        return

    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        if tag_name == 'Software':
            print(f"检测到编辑软件: {value}")
            if 'Photoshop' in str(value):
                print("⚠️ 图片可能被 Photoshop 编辑过！")
            return
    print("✅ 未发现明显编辑痕迹")

check_metadata_pure("/content/0.jpg")


检测到编辑软件: ç¾å¾ç§ç§-iOS-11.21.0


In [4]:
check_metadata_pure("/content/1.jpg")

检测到编辑软件: Android PD2352C_A_15.1.11.2.W10


In [4]:
check_metadata_pure("/content/8.jpg")

❌ 无 EXIF 信息，可能被清理


In [5]:
check_metadata_pure("/content/9.jpg")

✅ 未发现明显编辑痕迹


In [9]:
check_metadata_pure("/content/5.png")

❌ 无 EXIF 信息，可能被清理


In [3]:
check_metadata_pure("/content/7.jpg")

❌ 无 EXIF 信息，可能被清理


In [14]:
from PIL import Image
from PIL.ExifTags import TAGS
import re

def check_metadata_smart(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img.getexif()
    except Exception as e:
        print(f"❌ 无法读取图片：{e}")
        return

    if not exif_data or len(exif_data) == 0:
        print("❌ 无 EXIF 信息，可能为截图、社交平台转码或已清理元数据。")
        return

    # 提取 Software 字段
    software_value = None
    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        if tag_name == 'Software':
            software_value = str(value)
            break

    if not software_value:
        print("ℹ️ 未检测到 'Software' 字段，无法判断编辑来源。")
        return

    print(f"检测到编辑软件/系统标识: {software_value}")

    software_lower = software_value.lower()

    # ==== 1️⃣ 明确人工编辑类软件 ====
    editors = ['photoshop', 'meitu', '美图', 'snapseed', 'picsart', 'lightroom', 'pixlr', '醒图']
    if any(e in software_lower for e in editors):
        print("⚠️ 图片可能经过人工编辑或修图应用处理。")
        return

    # ==== 2️⃣ 系统压缩或社交应用重编码 ====
    if re.search(r'android|ios|huawei|oppo|vivo|miui|mms|wechat|qq|instagram|facebook', software_lower):
        if re.search(r'mms|彩信|pd\d{4}|_a_', software_lower):
            print("📨 检测到 Android 系统标识，可能是短信/彩信传输过程中的系统压缩。")
            print("✅ 这通常属于系统性重新编码，而非人工篡改。")
        elif 'wechat' in software_lower or 'wx' in software_lower:
            print("💬 检测到微信/社交平台压缩痕迹，可能是转发或自动压缩。")
            print("✅ 内容未被人工编辑，但画质可能降低。")
        else:
            print("⚙️ 检测到手机系统重新生成图像（可能是系统保存或分享过程）。")
            print("✅ 未发现明显人工编辑痕迹。")
        return

    # ==== 3️⃣ 未知软件但非空 ====
    print("✅ 未发现明显编辑痕迹（可能是原生拍摄或系统导出）。")


# 示例调用
check_metadata_smart("/content/3.jpg")


ℹ️ 未检测到 'Software' 字段，无法判断编辑来源。
